In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

### Read image into standard 2-d numpy array

In [ ]:
data = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
data = data.byteswap().newbyteorder()

### Show the original image

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('hubble-image.png')

### Background work

Measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

Evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()

Show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('hubble-background-image.png')

Evaluate the background noise as a 2-d array, same size as original image

In [ ]:
bkg_rms = bkg.rms()

Show the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('hubble-background-noise.png')

Subtract the background

In [ ]:
data_sub = data - bkg

### Detecting objects in image

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Plotting where each object is detected in the image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

max_l = 0

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6*objects['a'][i],
               height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    if objects['npix'][i] > objects['npix'][max_l]:
        max_l = i

plt.savefig('hubble-detected-objects.png')

### Obtaining fluxes from objects in image

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                    3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
print(f'Number of Sources Found: {len(objects)}')

Histogram fluxes

In [ ]:
fig, ax = plt.subplots(1, 1)
counts, bins, bars = ax.hist(flux)
ax.set_xlabel('Fluxes')
ax.set_ylabel('Frequency')
rects = ax.patches
for i in range(len(rects)):
    rect = rects[i]
    count = int(counts[i])
    if count != 0:
        ax.text(rect.get_x(), rect.get_height() + 100, f'{count}')

In [ ]:
mean, median, std = np.mean(flux), np.median(flux), np.std(flux)
print(f'Mean: {mean}')
print(f'Median: {median}')
print(f'Standard Deviation: {std}')

The largest outlier had a flux of 0.8073, at the approximate coordinates (2069, 1340) and is about 87 standard deviations away from the mean.

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
e = Ellipse(xy=(objects['x'][max_l], objects['y'][max_l]),
           width=6*objects['a'][max_l],
           height=6*objects['b'][max_l],
           angle=objects['theta'][max_l] * 180. / np.pi)
e.set_facecolor('none')
e.set_edgecolor('red')
ax.add_artist(e)

In [ ]:
from astropy.visualization import make_lupton_rgb

r = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits')
g = fits.getdata('hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits')
b = data
image = make_lupton_rgb(r, g, b, Q=5, stretch=0.05, filename='test.png')
plt.imshow(image, interpolation='nearest', vmin=m-s, vmax=m+s, origin='lower')
plt.savefig('hubble-colored-image.png')